# DeepAgents 探索ノートブック

このノートブックでは、LangChainエコシステムの**DeepAgents**について探索し、通常の`create_react_agent`との違いや、LangGraphシステムでの活用方法を学習します。

## 目次
1. [DeepAgentsとは？](#deepagentsとは)
2. [create_react_agentとの違い](#create_react_agentとの違い)
3. [基本的な使用例](#基本的な使用例)
4. [LangGraphとの統合](#langgraphとの統合)
5. [Test-Smithプロジェクトでの活用](#test-smithプロジェクトでの活用)

## 参考資料
- [DeepAgents 公式ドキュメント](https://docs.langchain.com/oss/python/deepagents/overview)
- [GitHub Repository](https://github.com/langchain-ai/deepagents)
- [API Reference](https://reference.langchain.com/python/deepagents/)
- [DeepAgents Blog](https://blog.langchain.com/deep-agents/)
- [DataCamp Tutorial](https://www.datacamp.com/tutorial/deep-agents)

---
## 1. DeepAgentsとは？

**DeepAgents**は、LangChainとLangGraph上に構築された、複雑なマルチステップタスクを処理できる高度なエージェントハーネスです。

### 主な特徴

#### 🎯 Planning Tool (計画ツール)
- タスクを個別のステップに分解する`write_todos`ツールを内蔵
- 複雑なタスクを管理可能な単位に分割

#### 📁 Filesystem Backend (ファイルシステム)
- コンテキストウィンドウのオーバーフローを防ぐためのファイルシステムツール
- `ls`, `read_file`, `write_file`, `edit_file`などのツールを提供
- 長期的なコンテキスト管理が可能

#### 🤖 Subagent Spawning (サブエージェント生成)
- 専門的なサブエージェントを生成する`task`ツール
- コンテキストの分離と専門化を実現

#### 📝 Detailed System Prompts (詳細なシステムプロンプト)
- Claude Codeなどのアプリケーションにインスパイアされた詳細な指示
- Few-shotプロンプトで特定の状況での振る舞いを学習

### インスピレーション元
- **Claude Code**: コーディングアシスタント
- **Deep Research**: 深い調査研究エージェント
- **Manus**: 複雑なタスク実行エージェント

---
## 2. create_react_agentとの違い

### 比較表

| 特徴 | create_react_agent | create_deep_agent |
|------|-------------------|------------------|
| **アーキテクチャ** | シンプルなReActループ | LangGraphベースの複雑なワークフロー |
| **Planning** | ❌ なし | ✅ write_todosツール内蔵 |
| **Subagents** | ❌ なし | ✅ taskツールで生成可能 |
| **Filesystem** | ❌ なし | ✅ ls, read_file, write_file, edit_file |
| **System Prompt** | 基本的 | 詳細で例示豊富 |
| **適用タスク** | 単純なツール呼び出し | 複雑なマルチステップタスク |
| **実行時間** | 短期 | 長期実行可能 |
| **コンテキスト管理** | メモリに依存 | ファイルシステムで永続化 |

### アーキテクチャの違い

#### create_react_agent (Shallow Agent)
```
User Query → LLM → Tool Call → LLM → Tool Call → ... → Final Answer
```
- 単純なループ構造
- 計画性がない
- コンテキストがすぐに溢れる

#### create_deep_agent (Deep Agent)
```
User Query → Planning (write_todos) → Subtask 1 (Subagent)
                                    → Subtask 2 (Subagent)
                                    → Subtask 3 (Subagent)
                                    → Synthesis → Final Answer
```
- 階層的な構造
- 計画的なタスク分解
- サブエージェントによる並列実行
- ファイルシステムでコンテキスト管理

---
## 3. 基本的な使用例

### インストール

In [ ]:
# DeepAgentsのインストール
# !pip install deepagents

# CLIサポート付きの場合
# !pip install deepagents-cli

### 基本的なDeepAgent作成

In [ ]:
from deepagents import create_deep_agent
from langchain_core.tools import tool

# カスタムツールの定義
@tool
def search_database(query: str) -> str:
    """データベースを検索するツール"""
    # ダミー実装
    return f"データベース検索結果: {query}に関する情報が見つかりました"

@tool
def analyze_data(data: str) -> str:
    """データを分析するツール"""
    # ダミー実装
    return f"データ分析結果: {data}の分析が完了しました"

# DeepAgentの作成
# 注意: デフォルトでclaude-sonnet-4-5を使用しますが、他のモデルも指定可能
agent = create_deep_agent(
    tools=[search_database, analyze_data],
    # model="gpt-4",  # モデルのカスタマイズも可能
)

print("DeepAgentが作成されました！")
print(f"利用可能なツール: {[tool.name for tool in [search_database, analyze_data]]}")

### DeepAgentの実行例

In [ ]:
# 複雑なタスクの実行
# 注意: 実際の実行にはAPIキーが必要です

# result = agent.invoke({
#     "messages": [
#         {
#             "role": "user",
#             "content": "データベースから最新の売上データを取得し、分析してレポートを作成してください"
#         }
#     ]
# })

# print(result)

---
## 4. LangGraphとの統合

DeepAgentsはLangGraph上に構築されているため、既存のLangGraphシステムとシームレスに統合できます。

### 統合パターン1: DeepAgentをノードとして使用

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated
import operator

# State定義
class ResearchState(TypedDict):
    query: str
    deep_research_results: Annotated[list[str], operator.add]
    final_report: str

# DeepAgentをラップするノード
def deep_research_node(state: ResearchState) -> ResearchState:
    """
    DeepAgentを使って深い調査を行うノード
    """
    # DeepAgentの実行
    # result = agent.invoke({
    #     "messages": [{"role": "user", "content": state["query"]}]
    # })
    
    # ダミーレスポンス
    result = f"DeepAgentによる調査結果: {state['query']}"
    
    return {
        "deep_research_results": [result]
    }

# LangGraphワークフローの構築
workflow = StateGraph(ResearchState)
workflow.add_node("deep_research", deep_research_node)
workflow.set_entry_point("deep_research")
workflow.set_finish_point("deep_research")

app = workflow.compile()
print("LangGraphワークフローが構築されました！")

### 統合パターン2: ハイブリッドアプローチ

Test-Smithのような既存のLangGraphシステムでは、特定のノードでDeepAgentを活用できます：

In [ ]:
# Test-Smithスタイルの統合例

class HybridState(TypedDict):
    query: str
    subtasks: list[str]
    subtask_results: Annotated[list[str], operator.add]
    final_synthesis: str

def master_planner_node(state: HybridState) -> HybridState:
    """マスタープランナー（従来のノード）"""
    subtasks = [
        "サブタスク1: データ収集",
        "サブタスク2: データ分析",
        "サブタスク3: レポート作成"
    ]
    return {"subtasks": subtasks}

def deep_agent_executor_node(state: HybridState) -> HybridState:
    """
    各サブタスクをDeepAgentで実行
    DeepAgentの強み（計画、ファイルシステム、サブエージェント）を活用
    """
    results = []
    
    for subtask in state["subtasks"]:
        # DeepAgentでサブタスクを実行
        # result = agent.invoke({"messages": [{"role": "user", "content": subtask}]})
        result = f"DeepAgentによる実行結果: {subtask}"
        results.append(result)
    
    return {"subtask_results": results}

def synthesizer_node(state: HybridState) -> HybridState:
    """最終合成ノード（従来のノード）"""
    synthesis = f"最終レポート: {len(state['subtask_results'])}個のサブタスク完了"
    return {"final_synthesis": synthesis}

# ハイブリッドワークフロー構築
hybrid_workflow = StateGraph(HybridState)
hybrid_workflow.add_node("master_planner", master_planner_node)
hybrid_workflow.add_node("deep_agent_executor", deep_agent_executor_node)
hybrid_workflow.add_node("synthesizer", synthesizer_node)

hybrid_workflow.set_entry_point("master_planner")
hybrid_workflow.add_edge("master_planner", "deep_agent_executor")
hybrid_workflow.add_edge("deep_agent_executor", "synthesizer")
hybrid_workflow.set_finish_point("synthesizer")

hybrid_app = hybrid_workflow.compile()
print("ハイブリッドワークフローが構築されました！")

---
## 5. Test-Smithプロジェクトでの活用

Test-Smithプロジェクトは既にLangGraphベースの高度なマルチエージェントシステムを持っています。DeepAgentsを活用する方法をいくつか提案します。

### 活用シナリオ1: 複雑なサブタスクの処理

現在のTest-Smithでは、各サブタスクは従来のノード（Strategic Planner → Searcher → Analyzer）で処理されています。
特に複雑なサブタスクについては、DeepAgentに委譲することで以下のメリットがあります：

- **自動計画**: サブタスクをさらに細かいステップに分解
- **ファイルシステム**: 長いコンテキストをファイルに保存
- **再帰的処理**: DeepAgentが内部でサブエージェントを生成

```python
# Test-Smithへの統合例
from src.graphs.deep_research_graph import DeepResearchGraphBuilder
from deepagents import create_deep_agent

def enhanced_subtask_processor(state: AgentState) -> AgentState:
    """
    複雑度が高いサブタスクはDeepAgentで処理
    """
    subtask = state["current_subtask"]
    complexity = estimate_complexity(subtask)  # 複雑度推定
    
    if complexity > THRESHOLD:
        # 複雑なサブタスク → DeepAgentに委譲
        deep_agent = create_deep_agent(tools=[...])
        result = deep_agent.invoke({"messages": [{"role": "user", "content": subtask}]})
    else:
        # シンプルなサブタスク → 従来の処理
        result = traditional_process(subtask)
    
    return {"subtask_results": [result]}
```

### 活用シナリオ2: 新しいグラフワークフローの作成

Test-Smithはマルチグラフアーキテクチャを採用しています。DeepAgentベースの新しいグラフを追加できます：

```python
# src/graphs/deepagent_research_graph.py

from src.graphs.base_graph import BaseGraphBuilder
from deepagents import create_deep_agent
from langgraph.graph import StateGraph

class DeepAgentResearchGraphBuilder(BaseGraphBuilder):
    def get_state_class(self) -> type:
        return DeepAgentState
    
    def build(self) -> StateGraph:
        workflow = StateGraph(DeepAgentState)
        
        # DeepAgentをメインエンジンとして使用
        workflow.add_node("deep_agent_research", self._deep_agent_node)
        workflow.set_entry_point("deep_agent_research")
        workflow.set_finish_point("deep_agent_research")
        
        return workflow.compile()
    
    def _deep_agent_node(self, state: DeepAgentState) -> DeepAgentState:
        # Test-Smithのツール（Tavily, ChromaDB）をDeepAgentに提供
        agent = create_deep_agent(
            tools=[tavily_search_tool, chromadb_retrieval_tool]
        )
        result = agent.invoke({"messages": [{"role": "user", "content": state["query"]}]})
        return {"report": result}
    
    def get_metadata(self) -> dict:
        return {
            "name": "deepagent_research",
            "description": "DeepAgentベースの自律的研究ワークフロー",
            "complexity": "High",
            "avg_time": "3-8 minutes"
        }
```

使用方法:
```bash
python main.py run "複雑な調査クエリ" --graph deepagent_research
```

### 活用シナリオ3: Code Investigation Graphの強化

Test-Smithの新しい`code_investigation`グラフは、DeepAgentと相性が良いです：

- **Code Retrieval**: DeepAgentのファイルシステムツールを活用
- **Dependency Analysis**: 複雑な依存関係追跡をDeepAgentに委譲
- **Recursive Code Exploration**: サブエージェントで各モジュールを探索

```python
# src/graphs/enhanced_code_investigation_graph.py

from deepagents import create_deep_agent
from langchain_core.tools import tool

@tool
def search_codebase(query: str) -> str:
    """ChromaDBでコードベースを検索"""
    # ChromaDB検索ロジック
    pass

@tool
def analyze_dependencies(file_path: str) -> str:
    """ファイルの依存関係を分析"""
    # 依存関係分析ロジック
    pass

def code_investigation_with_deepagent(state: CodeInvestigationState):
    # コード調査専用のDeepAgentを作成
    code_agent = create_deep_agent(
        tools=[search_codebase, analyze_dependencies],
        system_prompt="あなたはコードベース分析の専門家です..."
    )
    
    result = code_agent.invoke({
        "messages": [{"role": "user", "content": state["query"]}]
    })
    
    return {"investigation_report": result}
```

---
## まとめ

### DeepAgentの主な利点
1. **複雑なタスクの自動分解**: Planning toolで段階的に処理
2. **長期コンテキスト管理**: ファイルシステムでコンテキストウィンドウの制約を回避
3. **専門化と並列化**: サブエージェントで効率的な処理
4. **既存システムとの統合**: LangGraphベースなのでシームレス

### Test-Smithでの活用ポイント
- ✅ **ハイブリッドアプローチ**: 既存のノードと組み合わせる
- ✅ **新グラフ追加**: DeepAgentベースのワークフローを追加
- ✅ **Code Investigation強化**: ファイルシステムツールを活用
- ✅ **段階的導入**: まず特定のサブタスクから試す

### 次のステップ
1. `pip install deepagents`でインストール
2. このノートブックの例を試す
3. Test-Smithの特定グラフでDeepAgentを実験
4. パフォーマンスと複雑性のトレードオフを評価

---
## 実験セクション

以下のセルで自由に実験してください：

In [ ]:
# あなたの実験コードをここに記述